# Combining datasets to create our master dataset.

This notebook can be used to explore different options for combining our data. 

### Loading and preprocessing the HPSA dataset.

In [1]:
import time, os, math, numpy as np, pandas as pd, sklearn as sk, matplotlib as plt
# import sqlite3
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [2]:
# Load dataset for all Primary Care HPSA destinations with one record per facility
df_hpsa_pc = pd.read_csv("data/hpsa/BCD_HPSA_FCT_DET_PC.csv")

In [3]:
# q = """SELECT * 
#        FROM df_hpsa_pc
#        LIMIT 10;"""

# pysqldf(q)

In [4]:
print("Unique Cities: " + str(df_hpsa_pc["HPSA City"].nunique()))
print("Unique Counties " + str(df_hpsa_pc["Common County Name"].nunique()))

Unique Cities: 3319
Unique Counties 3210


In [6]:
df_hpsa_pc = df_hpsa_pc.rename(columns={'HPSA Score': 'PC_HPSA_Score'})

In [8]:
df_select_pc = df_hpsa_pc[["State Name",'HPSA City','Common County Name', 'County Equivalent Name',
                           'Designation Type', 'PC_HPSA_Score', 'PC MCTA Score', 'HPSA Status', 'HPSA Designation Date',
                          'HPSA Designation Last Update Date']]
df_select_pc.head()

,State Name,HPSA City,Common County Name,County Equivalent Name,Designation Type,PC_HPSA_Score,PC MCTA Score,HPSA Status,HPSA Designation Date,HPSA Designation Last Update Date
0,Ohio,NaN,"Lucas County, OH",Lucas,High Needs Geographic HPSA,15,18.0,Designated,01/31/1992,09/09/2021
1,Ohio,NaN,"Lucas County, OH",Lucas,High Needs Geographic HPSA,15,18.0,Designated,01/31/1992,09/09/2021
2,Ohio,NaN,"Lucas County, OH",Lucas,High Needs Geographic HPSA,15,18.0,Designated,01/31/1992,09/09/2021
3,Ohio,Caldwell,"Noble County, OH",Noble,Correctional Facility,6,0.0,Designated,08/25/1999,03/15/2022
4,Ohio,Edgerton,"Williams County, OH",Williams,Rural Health Clinic,14,15.0,Designated,12/16/2022,12/16/2022


In [10]:
df_select_counties = df_select_pc.groupby(["Common County Name"])["PC_HPSA_Score"].mean().reset_index()
df_select_cities = df_select_pc.groupby(["State Name", "HPSA City"])["PC_HPSA_Score"].mean().reset_index()

In [11]:
df_select_counties

,Common County Name,PC_HPSA_Score
0,"Abbeville County, SC",5.000000
1,"Acadia Parish, LA",12.500000
2,"Accomack County, VA",14.333333
3,"Ada County, ID",8.400000
4,"Adair County, IA",1.000000
...,...,...
3205,"Yuma County, CO",8.666667
3206,"Zapata County, TX",19.500000
3207,"Zavala County, TX",17.000000
3208,"Ziebach County, SD",6.923077


In [12]:
df_select_cities

,State Name,HPSA City,PC_HPSA_Score
0,Alabama,Albertville,8.500000
1,Alabama,Aliceville,21.000000
2,Alabama,Athens,0.000000
3,Alabama,Atmore,13.333333
4,Alabama,Bayou La Batre,18.000000
...,...,...,...
4009,Wyoming,Riverton,16.666667
4010,Wyoming,Rock Springs,11.000000
4011,Wyoming,Sundance,16.000000
4012,Wyoming,Thermopolis,8.000000


In [33]:
print(df_select_cities['HPSA City'].nunique())
print(len(df_select_cities))

3319
4014


In [34]:
df_select_cities["State_City"] = df_select_cities["State Name"] + ", " + df_select_cities["HPSA City"]
df_select_cities.head()

,State Name,HPSA City,PC_HPSA_Score,State_City
0,Alabama,Albertville,8.500000,"Alabama, Albertville"
1,Alabama,Aliceville,21.000000,"Alabama, Aliceville"
2,Alabama,Athens,0.000000,"Alabama, Athens"
3,Alabama,Atmore,13.333333,"Alabama, Atmore"
4,Alabama,Bayou La Batre,18.000000,"Alabama, Bayou La Batre"


### Loading FBI NIBRS data 

#### The clean_nibrs_df function in this notebook should be able to do the preliminary data cleaning on all of the Federal NIBRS datasets found here [https://ucr.fbi.gov/nibrs/2018/tables/data-tables#federal-table](https://ucr.fbi.gov/nibrs/2018/tables/data-tables#federal-table). Simply change the year in the URL to get the corresponding table. 

In [21]:
# Load 2019 FBI NIBRS data

df_nibrs_19 = pd.read_excel('data/nibrs/United_States_Offense_Type_by_Agency_2019.xls', skiprows=[0,1,2])

df_nibrs_19.head()

,State,Agency Type,Agency Name,Population1,Total\nOffenses,Crimes\nAgainst\nPersons,Crimes\nAgainst\nProperty,Crimes\nAgainst\nSociety,Crimes Against Persons,Unnamed: 9,...,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Assault\nOffenses,Aggravated\nAssault,...,Betting/\nWagering,Operating/\nPromoting/\nAssisting\nGambling,Gambling\nEquipment\nViolations,Sports\nTampering,Por-\nnography/\nObscene\nMaterial,Pros-\ntitution\nOffenses,Pros-\ntitution,Assisting or\nPromoting\nProstitution,Purchasing\nProstitution,Weapon\nLaw\nViolations
1,ALABAMA,Cities,Hoover,85670.0,4293.0,840.0,2801.0,652.0,811,68,...,0,0,0,0,2,1,1,0,0,67
2,ARIZONA,Cities,Apache Junction2,42531.0,NaN,430.0,NaN,487.0,383,71,...,0,0,0,0,6,0,0,0,0,36
3,NaN,NaN,Coolidge,13138.0,1155.0,230.0,684.0,241.0,214,61,...,0,0,0,0,1,0,0,0,0,18
4,NaN,NaN,Eagar,4897.0,102.0,36.0,29.0,37.0,21,7,...,0,0,0,0,0,0,0,0,0,1


In [30]:
def clean_nibrs_df(df, year):

    # Get missing column names
    names = df.iloc[0].tolist()
    missing_names = names[8:]
    # missing_names = [name.replace('\n', ' ').replace(' ', '_').replace('/_', '/').replace('-_', '').replace('__', '_') for name in missing_names]

    #Get column names and replace UnNamed columns with the corresponding name
    column_names = df.columns.to_list()
    column_names = column_names[:8]
    for name in missing_names:
        column_names.append(name)

    #Clean the column names up
    column_names = [name.replace('\n', ' ').replace(' ', '_').replace('/_', '/').replace('-_', '').replace('__', '_') for name in column_names]
    df.columns = column_names
    df = df.drop(df.index[0])
    
    # Replace NaN values in 'State' with the first non-NaN value using ffill
    df['State'] = df['State'].fillna(method='ffill')
    
    # Make sure just the first letter is capitalized
    df['State'] = df['State'].str.capitalize()

    # Replace NaN values in 'Agency_Type' with the first non-NaN value using ffill
    df['Agency_Type'] = df['Agency_Type'].fillna(method='ffill')
    
    #filter to the city level
    df = df[df["Agency_Type"]=="Cities"]
    
    df["Year"] = year
    
    df.drop(['Agency_Type'], axis=1, inplace=True)
    
    df.rename(columns = {"Agency_Name":"City"}, inplace=True)
    
    df = df.groupby(["State", "City"]).mean().reset_index()
    
    return df

In [31]:
cleaned_df_nibrs_19 = clean_nibrs_df(df_nibrs_19, 2019)

cleaned_df_nibrs_19.head()

,State,City,Population1,Total_Offenses,Crimes_Against_Persons,Crimes_Against_Property,Crimes_Against_Society,Assault_Offenses,Aggravated_Assault,Simple_Assault,...,Operating/Promoting/Assisting_Gambling,Gambling_Equipment_Violations,Sports_Tampering,Pornography/Obscene_Material,Prostitution_Offenses,Prostitution,Assisting_or_Promoting_Prostitution,Purchasing_Prostitution,Weapon_Law_Violations,Year
0,Alabama,Hoover,85670.0,4293.0,840.0,2801.0,652.0,811.0,68.0,504.0,...,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,67.0,2019.0
1,Arizona,Apache Junction2,42531.0,NaN,430.0,NaN,487.0,383.0,71.0,266.0,...,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,36.0,2019.0
2,Arizona,Coolidge,13138.0,1155.0,230.0,684.0,241.0,214.0,61.0,129.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,18.0,2019.0
3,Arizona,Eagar,4897.0,102.0,36.0,29.0,37.0,21.0,7.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2019.0
4,Arizona,Gilbert,253619.0,10589.0,1844.0,6144.0,2601.0,1642.0,157.0,1200.0,...,0.0,0.0,0.0,47.0,10.0,10.0,0.0,0.0,78.0,2019.0


In [61]:
print(cleaned_df_nibrs_19.shape)

(4168, 70)


In [35]:
cleaned_df_nibrs_19["State_City"] = cleaned_df_nibrs_19["State"] + ", " + cleaned_df_nibrs_19["City"]
cleaned_df_nibrs_19.head()

,State,City,Population1,Total_Offenses,Crimes_Against_Persons,Crimes_Against_Property,Crimes_Against_Society,Assault_Offenses,Aggravated_Assault,Simple_Assault,...,Gambling_Equipment_Violations,Sports_Tampering,Pornography/Obscene_Material,Prostitution_Offenses,Prostitution,Assisting_or_Promoting_Prostitution,Purchasing_Prostitution,Weapon_Law_Violations,Year,State_City
0,Alabama,Hoover,85670.0,4293.0,840.0,2801.0,652.0,811.0,68.0,504.0,...,0.0,0.0,2.0,1.0,1.0,0.0,0.0,67.0,2019.0,"Alabama, Hoover"
1,Arizona,Apache Junction2,42531.0,NaN,430.0,NaN,487.0,383.0,71.0,266.0,...,0.0,0.0,6.0,0.0,0.0,0.0,0.0,36.0,2019.0,"Arizona, Apache Junction2"
2,Arizona,Coolidge,13138.0,1155.0,230.0,684.0,241.0,214.0,61.0,129.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,18.0,2019.0,"Arizona, Coolidge"
3,Arizona,Eagar,4897.0,102.0,36.0,29.0,37.0,21.0,7.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2019.0,"Arizona, Eagar"
4,Arizona,Gilbert,253619.0,10589.0,1844.0,6144.0,2601.0,1642.0,157.0,1200.0,...,0.0,0.0,47.0,10.0,10.0,0.0,0.0,78.0,2019.0,"Arizona, Gilbert"


### Loading CDC Drug/Alcohol Related Deaths

In [45]:
df_cdc = pd.read_csv('data/cdc/Underlying Cause of Death, 2018-2021, Single Race (version 1).csv')

In [46]:
df_cdc.drop('Notes', axis=1, inplace=True)
df_cdc.head()

,County,County Code,Drug/Alcohol Induced,Drug/Alcohol Induced Code,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths,Population,Crude Rate,pct_tot,Years
0,"Autauga County, AL",1001,Drug-induced causes,D,Drug poisonings (overdose) Unintentional (X40-...,D1,20,226710,8.8,0.000088,2018-2021
1,"Autauga County, AL",1001,Alcohol-induced causes,A,All other alcohol-induced causes,A9,16,226710,Unreliable,0.000071,2018-2021
2,"Autauga County, AL",1001,All other non-drug and non-alcohol causes,O,All other non-drug and non-alcohol causes,O9,2398,226710,1057.7,0.010577,2018-2021
3,"Baldwin County, AL",1003,Drug-induced causes,D,Drug poisonings (overdose) Unintentional (X40-...,D1,185,909837,20.3,0.000203,2018-2021
4,"Baldwin County, AL",1003,Drug-induced causes,D,Drug poisonings (overdose) Suicide (X60-X64),D2,17,909837,Unreliable,0.000019,2018-2021


In [47]:
df_cdc.shape

(8244, 11)

In [48]:
df_cdc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8244 entries, 0 to 8243
Data columns (total 11 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   County                           8244 non-null   object 
 1   County Code                      8244 non-null   int64  
 2   Drug/Alcohol Induced             8244 non-null   object 
 3   Drug/Alcohol Induced Code        8244 non-null   object 
 4   Drug/Alcohol Induced Cause       8244 non-null   object 
 5   Drug/Alcohol Induced Cause Code  8244 non-null   object 
 6   Deaths                           8244 non-null   int64  
 7   Population                       8244 non-null   int64  
 8   Crude Rate                       8244 non-null   object 
 9   pct_tot                          8244 non-null   float64
 10  Years                            8244 non-null   object 
dtypes: float64(1), int64(3), object(7)
memory usage: 708.6+ KB


In [49]:
total = df_cdc.groupby(['County']).sum().reset_index()
total

,County,County Code,Drug/Alcohol Induced,Drug/Alcohol Induced Code,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths,Population,Crude Rate,pct_tot,Years
0,"Abbeville County, SC",135003,Drug-induced causesAlcohol-induced causesAll o...,DAO,Drug poisonings (overdose) Unintentional (X40-...,D1A9O9,1415,293313,22.5Unreliable1405.3,0.014473,2018-20212018-20212018-2021
1,"Acadia Parish, LA",88004,Drug-induced causesDrug-induced causesAlcohol-...,DDAO,Drug poisonings (overdose) Unintentional (X40-...,D1D4A9O9,3169,973764,Unreliable18.5Unreliable1268.1,0.013018,2018-20212018-20212018-20212018-2021
2,"Accomack County, VA",153003,Drug-induced causesAlcohol-induced causesAll o...,DAO,Drug poisonings (overdose) Unintentional (X40-...,D1A9O9,1981,390636,18.420.71482.2,0.015214,2018-20212018-20212018-2021
3,"Ada County, ID",112007,Drug-induced causesDrug-induced causesDrug-ind...,DDDDAAO,Drug poisonings (overdose) Unintentional (X40-...,D1D2D4D9A1A9O9,14385,13705181,14.12.1UnreliableUnreliableUnreliable14.9701.9,0.007347,2018-20212018-20212018-20212018-20212018-20212...
4,"Adair County, IA",19001,All other non-drug and non-alcohol causes,O,All other non-drug and non-alcohol causes,O9,432,28815,1499.2,0.014992,2018-2021
...,...,...,...,...,...,...,...,...,...,...,...
3134,"Yuma County, AZ",16108,Drug-induced causesDrug-induced causesAlcohol-...,DDAO,Drug poisonings (overdose) Unintentional (X40-...,D1D2A9O9,7924,3402916,21.2Unreliable10.6897.9,0.009314,2018-20212018-20212018-20212018-2021
3135,"Yuma County, CO",8125,All other non-drug and non-alcohol causes,O,All other non-drug and non-alcohol causes,O9,403,40027,1006.8,0.010068,2018-2021
3136,"Zapata County, TX",48505,All other non-drug and non-alcohol causes,O,All other non-drug and non-alcohol causes,O9,462,56449,818.4,0.008184,2018-2021
3137,"Zavala County, TX",48507,All other non-drug and non-alcohol causes,O,All other non-drug and non-alcohol causes,O9,521,45197,1152.7,0.011527,2018-2021


In [50]:
cdc_main = total[["County", "Deaths", "Population", "pct_tot"]]
cdc_main["Years"] = "2018-2021"
cdc_main

<ipython-input-50-55f920019d60>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cdc_main["Years"] = "2018-2021"


,County,Deaths,Population,pct_tot,Years
0,"Abbeville County, SC",1415,293313,0.014473,2018-2021
1,"Acadia Parish, LA",3169,973764,0.013018,2018-2021
2,"Accomack County, VA",1981,390636,0.015214,2018-2021
3,"Ada County, ID",14385,13705181,0.007347,2018-2021
4,"Adair County, IA",432,28815,0.014992,2018-2021
...,...,...,...,...,...
3134,"Yuma County, AZ",7924,3402916,0.009314,2018-2021
3135,"Yuma County, CO",403,40027,0.010068,2018-2021
3136,"Zapata County, TX",462,56449,0.008184,2018-2021
3137,"Zavala County, TX",521,45197,0.011527,2018-2021


In [52]:
cdc_main['pct_tot'] = cdc_main['Deaths']/cdc_main['Population']
cdc_main

<ipython-input-52-2ec0190d8883>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cdc_main['pct_tot'] = cdc_main['Deaths']/cdc_main['Population']


,County,Deaths,Population,pct_tot,Years
0,"Abbeville County, SC",1415,293313,0.004824,2018-2021
1,"Acadia Parish, LA",3169,973764,0.003254,2018-2021
2,"Accomack County, VA",1981,390636,0.005071,2018-2021
3,"Ada County, ID",14385,13705181,0.001050,2018-2021
4,"Adair County, IA",432,28815,0.014992,2018-2021
...,...,...,...,...,...
3134,"Yuma County, AZ",7924,3402916,0.002329,2018-2021
3135,"Yuma County, CO",403,40027,0.010068,2018-2021
3136,"Zapata County, TX",462,56449,0.008184,2018-2021
3137,"Zavala County, TX",521,45197,0.011527,2018-2021


In [53]:
col_names = ["County", "DrugOrAlc_Deaths", "Population", "DrugOrAlc_Deaths_pct", "Years"]
cdc_main.columns = col_names
cdc_main.head()

,County,DrugOrAlc_Deaths,Population,DrugOrAlc_Deaths_pct,Years
0,"Abbeville County, SC",1415,293313,0.004824,2018-2021
1,"Acadia Parish, LA",3169,973764,0.003254,2018-2021
2,"Accomack County, VA",1981,390636,0.005071,2018-2021
3,"Ada County, ID",14385,13705181,0.001050,2018-2021
4,"Adair County, IA",432,28815,0.014992,2018-2021


In [54]:
cdc_main.shape

(3139, 5)

In [57]:
df_select_counties.shape
print(df_select_counties.columns)

Index(['Common County Name', 'PC_HPSA_Score'], dtype='object')


In [58]:
df_select_counties.rename(columns={"Common County Name": "County"}, inplace=True)

# Combining Datasets on the City or the County Level

In [59]:
combined_df_counties = df_select_counties.merge(cdc_main, how='inner', on = "County")
combined_df_counties

,County,PC_HPSA_Score,DrugOrAlc_Deaths,Population,DrugOrAlc_Deaths_pct,Years
0,"Abbeville County, SC",5.000000,1415,293313,0.004824,2018-2021
1,"Acadia Parish, LA",12.500000,3169,973764,0.003254,2018-2021
2,"Accomack County, VA",14.333333,1981,390636,0.005071,2018-2021
3,"Ada County, ID",8.400000,14385,13705181,0.001050,2018-2021
4,"Adair County, IA",1.000000,432,28815,0.014992,2018-2021
...,...,...,...,...,...,...
3023,"Yuma County, AZ",6.877419,7924,3402916,0.002329,2018-2021
3024,"Yuma County, CO",8.666667,403,40027,0.010068,2018-2021
3025,"Zapata County, TX",19.500000,462,56449,0.008184,2018-2021
3026,"Zavala County, TX",17.000000,521,45197,0.011527,2018-2021


In [62]:
combined_df_cities = cleaned_df_nibrs_19.merge(df_select_cities, how='inner', on='State_City')
combined_df_cities.drop(['State Name', 'HPSA City'], axis=1, inplace=True)
combined_df_cities

,State,City,Population1,Total_Offenses,Crimes_Against_Persons,Crimes_Against_Property,Crimes_Against_Society,Assault_Offenses,Aggravated_Assault,Simple_Assault,...,Sports_Tampering,Pornography/Obscene_Material,Prostitution_Offenses,Prostitution,Assisting_or_Promoting_Prostitution,Purchasing_Prostitution,Weapon_Law_Violations,Year,State_City,PC_HPSA_Score
0,Arizona,Gilbert,253619.0,10589.0,1844.0,6144.0,2601.0,1642.0,157.0,1200.0,...,0.0,47.0,10.0,10.0,0.0,0.0,78.0,2019.0,"Arizona, Gilbert",18.0
1,Arizona,Somerton,16771.0,257.0,70.0,155.0,32.0,57.0,15.0,30.0,...,0.0,1.0,0.0,0.0,0.0,0.0,5.0,2019.0,"Arizona, Somerton",16.0
2,Arizona,Yuma,98769.0,7037.0,1110.0,4575.0,1352.0,965.0,309.0,536.0,...,0.0,30.0,2.0,0.0,2.0,0.0,93.0,2019.0,"Arizona, Yuma",0.0
3,Arkansas,Ash Flat,1095.0,86.0,19.0,52.0,15.0,19.0,5.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019.0,"Arkansas, Ash Flat",15.0
4,Arkansas,Augusta,1939.0,119.0,38.0,69.0,12.0,31.0,7.0,14.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2019.0,"Arkansas, Augusta",15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
878,Wisconsin,Sturtevant,6645.0,129.0,19.0,56.0,54.0,17.0,4.0,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2019.0,"Wisconsin, Sturtevant",6.0
879,Wisconsin,Waupun,11315.0,324.0,32.0,124.0,168.0,19.0,1.0,17.0,...,0.0,2.0,0.0,0.0,0.0,0.0,4.0,2019.0,"Wisconsin, Waupun",7.5
880,Wisconsin,Wausau,38507.0,2438.0,515.0,1007.0,916.0,387.0,110.0,245.0,...,0.0,21.0,2.0,1.0,1.0,0.0,53.0,2019.0,"Wisconsin, Wausau",10.0
881,Wisconsin,Whitewater,14997.0,666.0,101.0,296.0,269.0,68.0,22.0,42.0,...,0.0,8.0,0.0,0.0,0.0,0.0,9.0,2019.0,"Wisconsin, Whitewater",15.0


Note: We might want to try generating multiple years worth of data to expand the number of rows in the city dataset. 